In [2]:
pip install --upgrade ibm-watson

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
     |████████████████████████████████| 382 kB 17.9 MB/s eta 0:00:01
     |████████████████████████████████| 198 kB 48.8 MB/s eta 0:00:01
  Created wheel for ibm-watson: filename=ibm_watson-5.1.0-py3-none-any.whl size=375438 sha256=08b02cfc677e0d2cb4b056ec7a665a9610aa8fab1e8ccad8e7853918837607bc
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/c1/40/da/ba44de639af32759a5755cb79afedf309104b13fea09bbb5fc
  Created wheel for ibm-cloud-sdk-core: filename=ibm_cloud_sdk_core-3.10.0-py3-none-any.whl size=60913 sha256=d1f90d30554de0f90c189c0985d9bb

In [3]:
import pandas as pd
import json
import re
from pandas import json_normalize
from ibm_watson import AssistantV2
from IPython.display import display
from project_lib import Project
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [ ]:
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/master/src/main/python/cos_op.py
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/master/src/main/python/watson_assistant_func.py
!curl -O https://raw.githubusercontent.com/watson-developer-cloud/assistant-improve-recommendations-notebook/master/src/main/python/computation_func.py

In [5]:
# Import Cloud Object Storage related functions   
from cos_op import generate_link
from cos_op import generate_excel_measure
from cos_op import export_result_excel

# Import Watson Assistant related functions
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from watson_assistant_func import get_logs, get_assistant_definition

# Import Dataframe computation related functions
from computation_func import get_effective_df
from computation_func import get_coverage_df
from computation_func import chk_is_valid_node
from computation_func import format_data

In [6]:
#Variablen definieren fuer db_update
global kundennummer
kundennummer = 0
global postleitzahl
global ort
global strasse
global hausnummer

In [7]:
# Provide credentials to connect to assistant
authenticator = IAMAuthenticator('my_iam')
service = AssistantV2(version='2020-04-01', authenticator=authenticator)
service.set_service_url('my_url')

In [8]:
assistant_information = {'workspace_id' : '',
                        'skill_id' : 'my_skill',
                         'assistant_id' : 'my_assistant'}
assistant_id = 'my_assistant'

In [9]:
session_id = service.create_session(
 assistant_id = 'my_assistant'
).get_result()['session_id']

In [ ]:
print(json.dumps(session_id, indent=2))

In [11]:
#Dialog mit leerer Nachricht starten
response = service.message(
    assistant_id,
    session_id
).get_result()

In [12]:
# Mit leerem Wert initialisieren, um den Dialog zu starten
message_input = {
    'message_type:': 'text',
    'text': '',
    'options': {
        'return_context':True
    }
}

# Ausgabe des Dialogs anzeigen, sofern vorhanden. Unterstützt nur eine
# Textantwort.
if response['output']['generic']:
    if response['output']['generic'][0]['response_type'] == 'text':
        print(response['output']['generic'][0]['text'])
        
# Hauptschleife für Ein-/Ausgabe
while message_input['text'] != 'quit':

    # Nachricht an Assistent senden
    response = service.message(
        assistant_id,
        session_id,
        input = message_input,
        context={
            'global': {
                'system': {
                }
            },
            'skills': {
                'main skill': {
                    'user_defined': {
                        #'knr': 'knr',
                        #'plz': 'plz',
                        #'ort': 'ort',
                        #'strasse': 'strasse'
                    }
                }
            }
        }
    ).get_result()

    # Ausgabe des Dialogs anzeigen, sofern vorhanden. Unterstützt nur eine Textantwort.
    if response['output']['generic']:
        if response['output']['generic'][0]['response_type'] == 'text':
            print(response['output']['generic'][0]['text'])

    # Nächste Eingaberunde anfordern
    user_input = input('>> ')
    message_input = {
        'text': user_input,
        'options': {
        'return_context':True
        }
    }

Hallo. Ich bin Lea und freue mich, von Ihnen zu hören!
>> hallo
Wie kann ich Ihnen helfen?
>> ich bin umgezogen und möchte meine adresse ändern lassen
Dabei kann ich Ihnen gerne helfen! Möchten Sie Ihre Adresse ändern?
>> ja bitte
Geben Sie bitte Ihre Kundennummer ein!
>> 123456
Geben Sie bitte Ihre neue PLZ ein!
>> 1200
Ihre Eingabe 1200 wurde erkannt. Geben Sie bitte den Ort ein!
>> wien
Geben Sie bitte Ihre Strasse ein!
>> höchstädtplatz
Geben Sie bitte Ihre Hausnummer (inkl. Adresszusatz) ein!
>> 5/2
Vielen Dank für Ihre Eingaben. Wollen Sie die folgende Adresse für Ihre Kundennummer 123456 bestätigen? Höchstädtplatz 5/2 in 1200 Wien.
>> ja
Ihre Adresse wird  aktualisiert. Sie erhalten in Kürze ein Bestätigungsmail. Auf Wiedersehen.
>> dankeschön
Bis bald!
>> quit#
>> quit


In [ ]:
kundennummer = response['context']['skills']['main skill']['user_defined']['knr']
postleitzahl = response['context']['skills']['main skill']['user_defined']['plz']
ort = response['context']['skills']['main skill']['user_defined']['ort']
strasse = response['context']['skills']['main skill']['user_defined']['strasse']
hausnummer = response['context']['skills']['main skill']['user_defined']['hausnummer']

print(kundennummer, postleitzahl, ort, strasse, hausnummer)
print(response['context']['skills'])
print(response)

In [14]:
# delete session 
service.delete_session(
    assistant_id = assistant_id,
    session_id = session_id
)

In [15]:
import ibm_db

In [16]:
dsn_hostname = 'my_hostname'
dsn_uid = 'my_id'
dsn_pwd = 'my_pwd'

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [ ]:
#Create database connection
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

In [18]:
#Construct the query
updateQuery = "UPDATE KUNDENDATEN SET (ORT, STRASSE, POSTLEITZAHL, HAUSNUMMER) = ('{}', '{}', {}, '{}') WHERE KUNDENNUMMER = {};".format(ort, strasse, postleitzahl, hausnummer, kundennummer)

#execute the insert statement
updateStmt = ibm_db.exec_immediate(conn, updateQuery)